# Koodiklinikka Salary Survey 2024 - Data Cleaning

## Overview
This notebook cleans and prepares the **Koodiklinikka Salary Survey 2024** dataset for analysis. Koodiklinikka is a Finnish developer community, and their annual salary survey provides insights into compensation in the Finnish tech industry.

## Data Sources
- **Finnish survey** (`koodiklinikka_salary_survey_2024_fi.csv`): 685 responses in Finnish
- **English survey** (`koodiklinikka_salary_survey_2024_en.csv`): 54 responses in English

**Source:** Koodiklinikka. *Palkkakysely 2024.* https://koodiklinikka.github.io/palkkakysely/2024/

## Cleaning Steps
1. **Load & label** both datasets with source language
2. **Unify column names** (translate Finnish headers to English)
3. **Merge** datasets into a single DataFrame
4. **Filter** to employees only (exclude freelancers/entrepreneurs)
5. **Keep** only 100% full-time workers
6. **Remove** salary outliers (€1,800 - €15,000 monthly range)
7. **Filter** to Finnish locations only
8. **Translate** categorical values (gender, education, role, salary satisfaction)
9. **Create** normalized groupings (role_group, education_level, location_category)
10. **Export** cleaned dataset

## Output
- `koodiklinikka_salary_survey_2024_cleaned.csv`: Cleaned dataset ready for analysis
- Final dataset contains ~570 records with 21 columns

## Key Variables
| Column | Description |
|--------|-------------|
| `monthly_salary_eur` | Gross monthly salary in EUR |
| `annual_income_eur` | Total yearly income including bonuses |
| `experience_years` | Years of experience in the industry |
| `role_group` | Normalized role category (e.g., senior developer, data / ml / analytics) |
| `education_level` | Education level (phd, master, bachelor, vocational, secondary or less) |
| `location_category` | Capital region vs. rest of Finland |
| `salary_competitive` | Self-reported salary satisfaction (true, false, 50/50) |

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Ignore warnings related to NaN values
import warnings
warnings.filterwarnings("ignore")

# Detect if running on Kaggle
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

# Set input/output directories based on environment
if IS_KAGGLE:
    RAW_DIR = Path("/kaggle/input/finnish-tech-salary-survey-2024-data")
    PROCESSED_DIR = Path("/kaggle/working")
    print("Running on Kaggle")
else:
    RAW_DIR = Path("raw")
    PROCESSED_DIR = Path("processed")
    # Create processed directory if it doesn't exist
    PROCESSED_DIR.mkdir(exist_ok=True)
    print(f"Running locally - reading from '{RAW_DIR}/', saving to '{PROCESSED_DIR}/'")


Running locally - reading from 'raw/', saving to 'processed/'


## 1. Load Raw Data and Label Sources

Read both Finnish and English CSV files and add a `source_language` column to preserve provenance.

In [2]:
# Load Finnish data
df_fi = pd.read_csv(RAW_DIR / 'koodiklinikka_salary_survey_2024_fi.csv')
df_fi['source_language'] = 'fi'

# Load English data
df_en = pd.read_csv(RAW_DIR / 'koodiklinikka_salary_survey_2024_en.csv')
df_en['source_language'] = 'en'

print(f"Finnish dataset: {len(df_fi)} rows, {len(df_fi.columns)} columns")
print(f"English dataset: {len(df_en)} rows, {len(df_en.columns)} columns")


Finnish dataset: 683 rows, 27 columns
English dataset: 52 rows, 27 columns


## 2. Unify Column Names

Create a mapping dictionary to translate Finnish column headers into English.

In [3]:
# Finnish to English column mapping
# Note: Freelancer-specific columns and free text columns are excluded as they will be dropped later
fi_to_en_columns = {
    'Timestamp': 'timestamp',
    'Oletko palkansaaja vai laskuttaja?': 'employment_type',
    'Oletko siirtynyt palkansaajasta laskuttajaksi tai päinvastoin 1.10.2023 jälkeen?': 'switched_employment',
    'Ikä': 'age',
    'Sukupuoli': 'gender',
    'Työkokemus alalta (vuosina)': 'experience_years',
    'Koulutustaustasi': 'education',
    'Tulojen muutos viime vuodesta (%)': 'income_change_pct',
    'Työpaikka': 'company',
    'Missä kaupungissa työpaikkasi pääasiallinen toimisto sijaitsee?': 'office_city',
    'Millaisessa yrityksessä työskentelet?': 'company_type',
    'Työaika': 'work_time_percent',
    'Kuinka suuren osan ajasta teet lähityönä toimistolla?': 'office_time_percent',
    'Rooli / titteli': 'role',
    'Kuukausipalkka (brutto, euroina)': 'monthly_salary_eur',
    'Vuositulot (sis. bonukset, osingot yms, euroina)': 'annual_income_eur',
    'Onko palkkasi nykyroolissasi mielestäsi kilpailukykyinen?': 'salary_competitive',
    'Palautetta kyselystä ja ideoita ensi vuoden kyselyyn': 'survey_feedback',
    'source_language': 'source_language'
}

# Rename Finnish columns
df_fi = df_fi.rename(columns=fi_to_en_columns)


In [4]:
# English column mapping (standardize to lowercase snake_case)
# Note: Freelancer-specific columns and free text columns are excluded as they will be dropped later
en_to_standard_columns = {
    'Timestamp': 'timestamp',
    'Employee or entrepreneur': 'employment_type',
    'Have you switched from employment to entrepreneurship or vice versa after 1.10.2023?': 'switched_employment',
    'Age': 'age',
    'Gender': 'gender',
    'Relevant work experience from the industry (in years)': 'experience_years',
    'Education': 'education',
    'Change in income from last year (in %)': 'income_change_pct',
    'Company': 'company',
    'In which city is your office?': 'office_city',
    'What kind of a company you work in?': 'company_type',
    'Full time / part time': 'work_time_percent',
    'How much of your work time you spend in company office? (in %)': 'office_time_percent',
    'Role / title': 'role',
    'Monthly salary (gross, in EUR)': 'monthly_salary_eur',
    'Yearly income (incl. bonuses, etc; in EUR)': 'annual_income_eur',
    'Is your salary competitive?': 'salary_competitive',
    'Feedback of the survey': 'survey_feedback',
    'source_language': 'source_language'
}

# Rename English columns
df_en = df_en.rename(columns=en_to_standard_columns)

print(f"Finnish columns after rename: {list(df_fi.columns)}")
print(f"\nEnglish columns after rename: {list(df_en.columns)}")


Finnish columns after rename: ['timestamp', 'employment_type', 'switched_employment', 'age', 'gender', 'experience_years', 'education', 'income_change_pct', 'Montako vuotta olet tehnyt laskuttavaa työtä alalla?', 'Mitä palveluja tarjoat?', 'Tuntilaskutus (ALV 0%, euroina)', 'Vuosilaskutus (ALV 0%, euroina)', 'Hankitko asiakkaasi itse suoraan vai käytätkö välitysfirmojen palveluita?', 'Mistä asiakkaat ovat?', 'company', 'office_city', 'company_type', 'work_time_percent', 'office_time_percent', 'role', 'monthly_salary_eur', 'annual_income_eur', 'Vapaa kuvaus kokonaiskompensaatiomallista', 'salary_competitive', 'Vapaa sana', 'survey_feedback', 'source_language']

English columns after rename: ['timestamp', 'employment_type', 'switched_employment', 'age', 'gender', 'experience_years', 'education', 'income_change_pct', 'How many years have you worked as an entrepreneur in this industry?', 'What services do you offer?', 'Hourly rate (VAT 0%, in euros)', 'Yearly billing (VAT 0%, in euros)', '

## 3. Merge Datasets

Concatenate the Finnish and English DataFrames, ensuring columns align.

In [5]:
# Combine datasets
df = pd.concat([df_fi, df_en], ignore_index=True)

print(f"Combined dataset: {len(df)} rows, {len(df.columns)} columns")
print(f"\nLanguage distribution:")
print(df['source_language'].value_counts())


Combined dataset: 735 rows, 35 columns

Language distribution:
source_language
fi    683
en     52
Name: count, dtype: int64


In [6]:
# Normalize text columns: lowercase and trim whitespace
text_columns = ['gender', 'education', 'role', 'office_city', 'company_type', 'salary_competitive', 'company']

for col in text_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower().replace('nan', np.nan)

print("Text columns normalized (lowercase + trimmed)")


Text columns normalized (lowercase + trimmed)


## 4. Keep Only Employee Records

Filter rows where `employment_type` equals "Palkansaaja" (Finnish) or "Employee" (English).
Drop the freelancer/entrepreneur rows.

In [7]:
# Keep only employee records
employee_values = ['Palkansaaja', 'Employee']
df_employees = df[df['employment_type'].isin(employee_values)].copy()

dropped_freelancers = len(df) - len(df_employees)
print(f"Rows before filtering: {len(df)}")
print(f"Rows after filtering (employees only): {len(df_employees)}")
print(f"Dropped freelancer/entrepreneur rows: {dropped_freelancers}")


Rows before filtering: 735
Rows after filtering (employees only): 638
Dropped freelancer/entrepreneur rows: 97


## 5. Drop Freelancer-Specific Columns

Remove fields that only apply to freelancers.

In [8]:
# Define columns to drop (freelancer-specific, free text, and columns no longer needed)
columns_to_drop_fi = [
    'Montako vuotta olet tehnyt laskuttavaa työtä alalla?',
    'Mitä palveluja tarjoat?',
    'Tuntilaskutus (ALV 0%, euroina)',
    'Vuosilaskutus (ALV 0%, euroina)',
    'Hankitko asiakkaasi itse suoraan vai käytätkö välitysfirmojen palveluita?',
    'Mistä asiakkaat ovat?',
    'Vapaa sana',  # Free text column
    'Vapaa kuvaus kokonaiskompensaatiomallista'  # Compensation description
]

columns_to_drop_en = [
    'How many years have you worked as an entrepreneur in this industry?',
    'What services do you offer?',
    'Hourly rate (VAT 0%, in euros)',
    'Yearly billing (VAT 0%, in euros)',
    'Do you use agencies or find your clients yourself?',
    'What was left unasked that you want to answer to?',  # Free text column
    'Free description of your compensation model'  # Compensation description
]

# Additional columns to drop:
# - employment_type: all remaining rows are employees
# - timestamp: not needed for analysis
# - switched_employment: not needed for analysis
# - survey_feedback: free text not used
columns_to_drop = columns_to_drop_fi + columns_to_drop_en + [
    'employment_type', 
    'timestamp', 
    'switched_employment', 
    'survey_feedback'
]

# Drop columns
df_employees = df_employees.drop(columns=columns_to_drop, errors='ignore')

print(f"Columns after dropping unused fields: {len(df_employees.columns)}")
print(f"Remaining columns: {list(df_employees.columns)}")


Columns after dropping unused fields: 16
Remaining columns: ['age', 'gender', 'experience_years', 'education', 'income_change_pct', 'company', 'office_city', 'company_type', 'work_time_percent', 'office_time_percent', 'role', 'monthly_salary_eur', 'annual_income_eur', 'salary_competitive', 'source_language', 'Where are your clients from?']


## 6. Restrict to Full-Time Workers

Convert `work_time_percent` to numeric and retain only rows equal to 100.

In [9]:
# Convert work_time_percent to numeric
df_employees['work_time_percent'] = pd.to_numeric(df_employees['work_time_percent'], errors='coerce')

# Filter for full-time workers (100%)
df_fulltime = df_employees[df_employees['work_time_percent'] == 100].copy()

dropped_parttime = len(df_employees) - len(df_fulltime)
print(f"Rows before filtering: {len(df_employees)}")
print(f"Rows after filtering (100% full-time only): {len(df_fulltime)}")
print(f"Dropped part-time/over-time rows: {dropped_parttime}")

# Drop work_time_percent column (no longer needed after filtering)
df_fulltime = df_fulltime.drop(columns=['work_time_percent'])
print(f"\nDropped 'work_time_percent' column (all rows are 100%)")


Rows before filtering: 638
Rows after filtering (100% full-time only): 575
Dropped part-time/over-time rows: 63

Dropped 'work_time_percent' column (all rows are 100%)


## 7. Convert Numeric Fields

Cast `monthly_salary_eur` and `annual_income_eur` to floats and filter unrealistic outliers.

In [10]:
# Convert salary fields to numeric
df_fulltime['monthly_salary_eur'] = pd.to_numeric(df_fulltime['monthly_salary_eur'], errors='coerce')
df_fulltime['annual_income_eur'] = pd.to_numeric(df_fulltime['annual_income_eur'], errors='coerce')
df_fulltime['experience_years'] = pd.to_numeric(df_fulltime['experience_years'], errors='coerce')
df_fulltime['income_change_pct'] = pd.to_numeric(df_fulltime['income_change_pct'], errors='coerce')
df_fulltime['office_time_percent'] = pd.to_numeric(df_fulltime['office_time_percent'], errors='coerce')

print("Monthly salary statistics:")
print(df_fulltime['monthly_salary_eur'].describe())


Monthly salary statistics:
count      575.000000
mean      5767.508313
std       1635.551392
min          0.000000
25%       4760.000000
50%       5624.000000
75%       6505.500000
max      14200.000000
Name: monthly_salary_eur, dtype: float64


In [11]:
# Define realistic salary bounds (Finnish minimum wage ~1800 EUR, max 15000 EUR)
MIN_MONTHLY_SALARY = 1800
MAX_MONTHLY_SALARY = 15000

# Filter outliers
df_clean = df_fulltime[
    (df_fulltime['monthly_salary_eur'] >= MIN_MONTHLY_SALARY) & 
    (df_fulltime['monthly_salary_eur'] <= MAX_MONTHLY_SALARY)
].copy()

dropped_outliers = len(df_fulltime) - len(df_clean)
print(f"Rows before outlier filtering: {len(df_fulltime)}")
print(f"Rows after outlier filtering: {len(df_clean)}")
print(f"Dropped outlier rows: {dropped_outliers}")
print(f"\nMonthly salary range: {MIN_MONTHLY_SALARY} - {MAX_MONTHLY_SALARY} EUR")


Rows before outlier filtering: 575
Rows after outlier filtering: 572
Dropped outlier rows: 3

Monthly salary range: 1800 - 15000 EUR


In [12]:
# Gender translation and normalization (values already lowercased)
def translate_gender(gender):
    """Translate and normalize gender values to male, female, or other."""
    if pd.isna(gender):
        return None
    
    # Male variations
    if gender in ['mies', 'male', 'm', 'miesoletettu']:
        return 'male'
    
    # Female variations
    if gender in ['nainen', 'female', 'f', 'woman']:
        return 'female'
    
    return 'other'

# Apply gender translation
df_clean['gender'] = df_clean['gender'].apply(translate_gender)

print("Gender distribution after translation:")
print(df_clean['gender'].value_counts(dropna=False))


Gender distribution after translation:
gender
male      449
female     67
None       43
other      13
Name: count, dtype: int64


In [13]:
# Salary competitive translation (values already lowercased)
def translate_salary_competitive(value):
    """Translate salary_competitive to true, false, or 50/50."""
    if pd.isna(value):
        return None
    
    # Yes / True
    if value in ['kyllä', 'yes', 'true']:
        return 'true'
    
    # No / False
    if value in ['ei', 'no', 'false']:
        return 'false'
    
    # Everything else is 50/50
    return '50/50'

# Apply translation
df_clean['salary_competitive'] = df_clean['salary_competitive'].apply(translate_salary_competitive)

print("Salary competitive distribution after translation:")
print(df_clean['salary_competitive'].value_counts(dropna=False))


Salary competitive distribution after translation:
salary_competitive
true     378
false    141
50/50     39
None      14
Name: count, dtype: int64


## 8. Translate Categorical Values

Create translation mappings for Finnish values in `education` and `role` columns.

In [14]:
# Education translation mapping (Finnish to English) - all keys lowercase since data is normalized
education_translation = {
    # Master's level
    'di': 'msc (engineering)',
    'diplomi-insinööri': 'msc (engineering)',
    'fm': 'msc (science)',
    'fm (tietojenkäsittelytiede)': 'msc (computer science)',
    'maisteri': "master's degree",
    'tekniikan maisteritutkinto': 'msc (technology)',
    'tietojenkäsittelytieteen maisteri': 'msc (computer science)',
    'ylempi korkeakoulututkinto': "master's degree",
    'ylempi korkeakoulututkinto-': "master's degree",
    'kauppatieteiden maisteri': 'msc (business)',
    # Bachelor's level (AMK)
    'amk': "bachelor's (amk)",
    'ammattikorkeakoulu': "bachelor's (amk)",
    'amk insinööri': "bachelor's engineering (amk)",
    'insinööri (amk)': "bachelor's engineering (amk)",
    'insinööri-amk': "bachelor's engineering (amk)",
    'amk-insinööri': "bachelor's engineering (amk)",
    'it-tradenomi': "bachelor's it (amk)",
    'tradenomi': "bachelor's business (amk)",
    'tradenomi amk': "bachelor's business (amk)",
    # Bachelor's level (University)
    'kandi': "bachelor's degree",
    'luonnontieteiden kandidaatti': 'bsc (natural sciences)',
    'tkt kandi': 'bsc (computer science)',
    'tietotekniikan kandidaatti': 'bsc (computer science)',
    'tietojenkäsittelytieteen kandidaatti': 'bsc (computer science)',
    # PhD level
    'tohtori': 'phd',
    'ytt': 'phd (social sciences)',
    # Secondary/Vocational
    'ylioppilas': 'high school (ylioppilas)',
    'lukio': 'high school',
    'ammattikoulu': 'vocational school',
    'datanomi': 'vocational it diploma',
    'peruskoulu': 'basic education',
    # Other
    'tietojenkäsittelytiede': 'computer science (unspecified)',
    'yliopisto': 'university (unspecified)',
}

def translate_education(edu):
    """Translate education value from Finnish to English."""
    if pd.isna(edu):
        return None
    # Direct match (data already lowercase)
    if edu in education_translation:
        return education_translation[edu]
    # Partial match for common patterns
    if 'diplomi-insinööri' in edu or edu == 'di':
        return 'msc (engineering)'
    if 'amk' in edu and 'insinööri' in edu:
        return "bachelor's engineering (amk)"
    if 'amk' in edu:
        return "bachelor's (amk)"
    if 'maisteri' in edu or 'master' in edu:
        return "master's degree"
    if 'kandidaatti' in edu or 'bachelor' in edu:
        return "bachelor's degree"
    if 'lukio' in edu or 'ylioppilas' in edu:
        return 'high school'
    if 'tohtori' in edu or 'phd' in edu:
        return 'phd'
    # Return original if no match
    return edu

# Apply translation
df_clean['education_en'] = df_clean['education'].apply(translate_education)

print("Education translation sample:")
print(df_clean[['education', 'education_en']].drop_duplicates().head(20))


Education translation sample:
                                            education  \
0                                          ylioppilas   
2                          tietotekniikan kandidaatti   
4                                                  di   
5                                        it-tradenomi   
6                                                 NaN   
7                         fm (tietojenkäsittelytiede)   
9                                     insinööri (amk)   
12                                                ytt   
13                         tekniikan maisteritutkinto   
14                                  diplomi-insinööri   
15                       luonnontieteiden kandidaatti   
16                             amk software developer   
19                             tietojenkäsittelytiede   
23  luonnontieteiden kandidaatti, tietojenkäsittel...   
24                                 ammattikorkeakoulu   
26                                      amk insinööri   
2

In [15]:
# Role translation - most roles are already in English, just clean up Finnish ones (keys lowercase)
role_translation = {
    'tunkkaaja': 'developer',
    'pilviarkkitehti': 'cloud architect',
    'tietoturva-asiantuntija': 'security specialist',
}

def translate_role(role):
    """Translate role from Finnish to English if needed."""
    if pd.isna(role):
        return None
    if role in role_translation:
        return role_translation[role]
    return role

# Apply role translation
df_clean['role_en'] = df_clean['role'].apply(translate_role)

print("Role translation applied")
print(df_clean[['role', 'role_en']].drop_duplicates().head(10))


Role translation applied
                                            role  \
0                     senior fullstack developer   
2                      site reliability engineer   
3                            fullstack developer   
4                                           lead   
5   software developer, fullstack clojure/script   
6                      technical project manager   
7                            software specialist   
9                               senior developer   
12                            software developer   
13                      test automation engineer   

                                         role_en  
0                     senior fullstack developer  
2                      site reliability engineer  
3                            fullstack developer  
4                                           lead  
5   software developer, fullstack clojure/script  
6                      technical project manager  
7                            software special

## 9. Normalize Roles and Education

Create normalized grouping columns for roles and education levels.

In [16]:
# Role grouping function (data already lowercase)
def classify_role_group(role):
    """Classify role into standardized groups based on keywords."""
    if pd.isna(role):
        return 'other'
    
    # Management / Leadership
    if any(kw in role for kw in ['cto', 'ceo', 'director', 'vp ', 'head of', 'manager', 'johtaja']):
        return 'management / leadership'
    
    # Lead Developer (check before general developer category)
    if 'lead' in role and any(kw in role for kw in ['developer', 'engineer', 'dev', 'software', 'tech']):
        return 'lead developer'
    
    # Senior Developer
    if 'senior' in role and any(kw in role for kw in ['developer', 'engineer', 'dev', 'software', 'programmer']):
        return 'senior developer'
    
    # Junior Developer
    if 'junior' in role and any(kw in role for kw in ['developer', 'engineer', 'dev', 'software', 'programmer']):
        return 'junior developer'
    
    # Data / ML / Analytics
    if any(kw in role for kw in ['data', 'ml ', 'machine learning', 'ai ', 'analytics', 'scientist', 'bi ']):
        return 'data / ml / analytics'
    
    # Design / UX
    if any(kw in role for kw in ['ux', 'ui', 'design', 'designer']):
        return 'design / ux'
    
    # Product / Project
    if any(kw in role for kw in ['product', 'project', 'scrum', 'agile', 'po ', 'owner']):
        return 'product / project'
    
    # DevOps / Infrastructure
    if any(kw in role for kw in ['devops', 'sre', 'infrastructure', 'platform', 'cloud', 'reliability']):
        return 'devops / infrastructure'
    
    # Architect
    if 'architect' in role:
        return 'architect'
    
    # Software Developer / Engineer (default for dev roles - mid-level/unspecified)
    if any(kw in role for kw in ['developer', 'engineer', 'programmer', 'software', 'fullstack', 
                                  'full stack', 'frontend', 'backend', 'full-stack', 'mobile',
                                  'web', 'specialist', 'consultant', 'coder']):
        return 'software developer / engineer'
    
    # QA / Testing
    if any(kw in role for kw in ['qa', 'test', 'quality']):
        return 'qa / testing'
    
    # Security
    if any(kw in role for kw in ['security', 'tietoturva']):
        return 'security'
    
    return 'other'

# Apply role grouping
df_clean['role_group'] = df_clean['role_en'].apply(classify_role_group)

print("Role group distribution:")
print(df_clean['role_group'].value_counts())


Role group distribution:
role_group
software developer / engineer    240
senior developer                 114
other                             48
management / leadership           36
lead developer                    36
devops / infrastructure           32
data / ml / analytics             27
architect                         25
product / project                  7
design / ux                        5
junior developer                   2
Name: count, dtype: int64


In [17]:
# Education level classification (data already lowercase)
def classify_education_level(edu):
    """Classify education into standardized levels."""
    if pd.isna(edu):
        return 'unknown'
    
    # PhD
    if 'phd' in edu or 'tohtori' in edu or 'doctor' in edu:
        return 'phd'
    
    # Master's
    if any(kw in edu for kw in ['msc', 'master', 'diplomi-insinööri', 'di', 'fm', 'maisteri', 'ylempi']):
        return 'master'
    
    # Bachelor's
    if any(kw in edu for kw in ['bachelor', 'bsc', 'amk', 'kandidaatti', 'kandi', 'tradenomi', 'insinööri']):
        return 'bachelor'
    
    # Vocational
    if any(kw in edu for kw in ['vocational', 'datanomi', 'ammattikoulu', 'ammatti']):
        return 'vocational'
    
    # Secondary or less
    if any(kw in edu for kw in ['high school', 'lukio', 'ylioppilas', 'peruskoulu', 'basic']):
        return 'secondary or less'
    
    return 'other'

# Apply education level classification
df_clean['education_level'] = df_clean['education_en'].apply(classify_education_level)

print("Education level distribution:")
print(df_clean['education_level'].value_counts())


Education level distribution:
education_level
master               216
bachelor             191
other                 62
secondary or less     48
unknown               33
vocational            17
phd                    5
Name: count, dtype: int64


## 10. Add Location Categories

Create a location category column (Helsinki capital region vs. other cities).

In [18]:
# Location categorization (data already lowercase)
# Finnish cities list for filtering
finnish_cities = [
    'helsinki', 'espoo', 'vantaa', 'pk-seutu', 'capital region',
    'tampere', 'turku', 'oulu', 'jyväskylä', 'lahti', 'kuopio',
    'pori', 'joensuu', 'lappeenranta', 'rovaniemi', 'vaasa',
    'kotka', 'hämeenlinna', 'kouvola', 'seinäjoki', 'mikkeli',
    'finland', 'suomi'
]

def is_in_finland(city):
    """Check if the city is in Finland."""
    if pd.isna(city):
        return False
    return any(kw in city for kw in finnish_cities)

def categorize_location(city):
    """Categorize city into Capital Region vs. Rest of Finland."""
    if pd.isna(city):
        return None
    
    # Helsinki Capital Region (PK-seutu)
    if any(kw in city for kw in ['helsinki', 'espoo', 'vantaa', 'pk-seutu', 'capital region']):
        return 'capital region'
    
    return 'rest of finland'

# Filter to only Finnish locations
rows_before = len(df_clean)
df_clean = df_clean[df_clean['office_city'].apply(is_in_finland)].copy()
dropped_non_finland = rows_before - len(df_clean)

print(f"Dropped {dropped_non_finland} rows with non-Finnish locations")

# Apply location categorization
df_clean['location_category'] = df_clean['office_city'].apply(categorize_location)

print("\nLocation category distribution:")
print(df_clean['location_category'].value_counts())


Dropped 33 rows with non-Finnish locations

Location category distribution:
location_category
capital region     320
rest of finland    219
Name: count, dtype: int64


## 11. Export Clean Dataset

Select final columns and save the cleaned CSV file.

In [19]:
# Define final columns to keep
final_columns = [
    # Identifiers and demographics
    'source_language',
    'age',
    'gender',
    'experience_years',
    
    # Salary data
    'monthly_salary_eur',
    'annual_income_eur',
    'income_change_pct',
    
    # Employment info
    'company',
    'company_type',
    
    # Role - original, translated, and normalized
    'role',
    'role_en',
    'role_group',
    
    # Education - original, translated, and normalized
    'education',
    'education_en',
    'education_level',
    
    # Location
    'office_city',
    'location_category',
    'office_time_percent',
    
    # Perception
    'salary_competitive',
]

# Create final dataset
df_final = df_clean[final_columns].copy()

print(f"Final dataset: {len(df_final)} rows, {len(df_final.columns)} columns")


Final dataset: 539 rows, 19 columns


In [20]:
# Summary statistics
print("=" * 60)
print("DATASET SUMMARY")
print("=" * 60)
print(f"\nTotal records: {len(df_final)}")
print(f"\nLanguage distribution:")
print(df_final['source_language'].value_counts())
print(f"\nSalary statistics:")
print(df_final['monthly_salary_eur'].describe())
print(f"\nRole group distribution:")
print(df_final['role_group'].value_counts())
print(f"\nEducation level distribution:")
print(df_final['education_level'].value_counts())
print(f"\nLocation category distribution:")
print(df_final['location_category'].value_counts())


DATASET SUMMARY

Total records: 539

Language distribution:
source_language
fi    495
en     44
Name: count, dtype: int64

Salary statistics:
count      539.000000
mean      5775.664712
std       1506.852552
min       1890.000000
25%       4800.000000
50%       5690.000000
75%       6514.500000
max      12500.000000
Name: monthly_salary_eur, dtype: float64

Role group distribution:
role_group
software developer / engineer    225
senior developer                 107
other                             47
lead developer                    34
devops / infrastructure           32
management / leadership           30
data / ml / analytics             25
architect                         25
product / project                  7
design / ux                        5
junior developer                   2
Name: count, dtype: int64

Education level distribution:
education_level
master               210
bachelor             176
other                 57
secondary or less     44
unknown               31

In [21]:
# Document dropped rows/columns
print("=" * 60)
print("DATA CLEANING DOCUMENTATION")
print("=" * 60)
print(f"\n1. Original combined rows: {len(df)}")
print(f"2. After keeping employees only: {len(df_employees)} (dropped {len(df) - len(df_employees)} freelancers)")
print(f"3. After keeping 100% full-time: {len(df_fulltime)} (dropped {dropped_parttime} part-time)")
print(f"4. After salary outlier filtering: (dropped {dropped_outliers} rows outside {MIN_MONTHLY_SALARY}-{MAX_MONTHLY_SALARY} EUR)")
print(f"5. After Finnish location filtering: (dropped {dropped_non_finland} non-Finnish locations)")
print(f"6. Final dataset: {len(df_final)} rows, {len(df_final.columns)} columns")


DATA CLEANING DOCUMENTATION

1. Original combined rows: 735
2. After keeping employees only: 638 (dropped 97 freelancers)
3. After keeping 100% full-time: 575 (dropped 63 part-time)
4. After salary outlier filtering: (dropped 3 rows outside 1800-15000 EUR)
5. After Finnish location filtering: (dropped 33 non-Finnish locations)
6. Final dataset: 539 rows, 19 columns


In [22]:
# Save cleaned dataset
output_file = PROCESSED_DIR / 'koodiklinikka_salary_survey_2024_cleaned.csv'
df_final.to_csv(output_file, index=False)
print(f"\nCleaned dataset saved to: {output_file.absolute()}")

# Display first few rows
df_final.head()



Cleaned dataset saved to: /Users/elar.saks/Desktop/finnish-tech-salary-survey/data/processed/koodiklinikka_salary_survey_2024_cleaned.csv


,source_language,age,gender,experience_years,monthly_salary_eur,annual_income_eur,income_change_pct,company,company_type,role,role_en,role_group,education,education_en,education_level,office_city,location_category,office_time_percent,salary_competitive
0,fi,46-50,male,27.0,7250.0,96000.0,3.0,NaN,konsulttitalossa,senior fullstack developer,senior fullstack developer,senior developer,ylioppilas,high school (ylioppilas),secondary or less,"pk-seutu (helsinki, espoo, vantaa)",capital region,80.0,true
2,fi,31-35,male,11.0,5100.0,65200.0,2.0,NaN,"yrityksessä, jossa softa on tukeva toiminto (e...",site reliability engineer,site reliability engineer,devops / infrastructure,tietotekniikan kandidaatti,bsc (computer science),bachelor,"pk-seutu (helsinki, espoo, vantaa)",capital region,10.0,true
3,fi,36-40,male,12.0,5850.0,105000.0,0.0,NaN,konsulttitalossa,fullstack developer,fullstack developer,software developer / engineer,ylioppilas,high school (ylioppilas),secondary or less,"pk-seutu (helsinki, espoo, vantaa)",capital region,5.0,true
4,fi,41-45,male,15.0,6800.0,81600.0,NaN,NaN,"tuotetalossa, jonka core-bisnes on softa",lead,lead,other,di,msc (engineering),master,tampere,rest of finland,30.0,true
5,fi,41-45,female,9.0,4730.0,61500.0,0.0,NaN,"tuotetalossa, jonka core-bisnes on softa","software developer, fullstack clojure/script","software developer, fullstack clojure/script",software developer / engineer,it-tradenomi,bachelor's it (amk),bachelor,helsinki & tampere,capital region,20.0,true
